# Perform a looping with a quadrotor

# Goal of the project

The goal of this project is to control a 2D quadrotor to get it to perform acrobatic moves. The controller will be designed using an SQP solver.

Please submit your code as a runnable Jupyter Notebook.

## 2D quadrotor

The quadrotor is depicted in the following figure
<img src='quadrotor.png' width="300">


The quadrotor model is written as
$$\begin{align} 
\dot{p_x} &= v_x\\
m \dot{v}_x &= - (u_1 + u_2) \sin \theta \\ 
\dot{p_y} &= v_y\\
m \dot{v}_y &= (u_1 + u_2) \cos \theta  - m g\\
\dot{\theta} &= \omega\\
I \dot{\omega} &= r (u_1 - u_2) \end{align}$$
where $p_x$ is the horizontal and $p_y$ the vertical positions of the quadrotor and $\theta$ is its orientation with respect to the horizontal plane. $v_x$ and $v_y$ are the linear velocities and $\omega$ is the angular velocity of the robot. $u_1$ and $u_2$ are the forces produced by the rotors (our control inputs). $m$ is the quadrotor mass, $I$ its moment of inertia (a scalar), $r$ is the distance from the center of the robot frame to the propellers and $g$ is the gravity constant. To denote the entire state, we will write $x = [p_x, v_x, p_y, v_y, \theta, \omega]^T$ - we will also write $u = [u_1, u_2]^T$.

The module ```quadrotor.py``` defines the problem and provides all the useful information about the robot and methods to simulate and animate it as shown below.

You can access the different parameters of the model in the following way:

In [1]:
import quadrotor

print("Mass    =", quadrotor.MASS)
print("Length  =", quadrotor.LENGTH)
print("Inertia =", quadrotor.INERTIA)
print("Dt      =", quadrotor.DT)
print("state size   =", quadrotor.DIM_STATE)
print("control size =", quadrotor.DIM_CONTROL)

Mass    = 0.45
Length  = 0.14
Inertia = 0.09
Dt      = 0.04
state size   = 6
control size = 2




## Part 1 - Setting up the trajectory optimization problem (50 points)
1. Discretize the system dynamics using Euler's method seen in class - write the time discretization step as $\Delta t$ (use symbols not numbers for the mass, etc)
2. We would like the quadrotor to perfom a looping (i.e. a back or front flip). Find and implement a suitable cost function to perform a looping and add constraints to maintain the thrust of each rotor between $0$ and $10$. Solve the problem using your own implementation of a SQP with a long horizon to ensure that you can do a looping.
3. Show plots of all the states and controls of the robot as a function of time. Plot the value of $\alpha$ (during line search) at each iteration of the solver as well as the amount of constraint violation per iteration and the cost per iteration of the solver (also useful to help you debug).
5. Describe your design choices (in a concise manner) in the report.

To visualize the trajectory, use the ```quadrotor.animate_robot``` function and show the animation (show the plots in your report).

### Tips to implement a SQP solver
1. Write down the algorithm (in words not in code), i.e. write all the steps you need to take
2. Write the gradient of the running cost at a given guess $\bar{x} = [{\bar{p}_x}_0, {\bar{v}_x}_0, {\bar{p}_y}_0, {\bar{v}_{y}}_0, \bar{\theta}_0, \bar{\omega}_0, {\bar{u}_1}_0, {\bar{u}_2}_0, \dots, {\bar{p}_x}_N, {\bar{v}_x}_N, {\bar{p}_y}_N, {\bar{v}_{y}}_N, \bar{\theta}_N, \bar{\omega}_N, {\bar{u}_1}_N, {\bar{u}_2}_N,]^T$, i.e. for a given value of these variables and implement a function that computes it
3. Write the Hessian of the running cost at a given guess $\bar{x}$ and implement a function that computes it
4. Assume that the Hessian of the constraints is 0 (i.e. ignore the second order derivatives of the constraints)
5. Write a linear approximation of the constraints at a given guess $\bar{x}$ in a form $G(\bar{x}) \Delta x = g(\bar{x})$ (don't forget the constant terms in g) where $\Delta x$ represents a small variation around $\bar{x}$ and implement a function that computes both $G$ and $g$.
6. Use these functions to setup the inner QP of the SQP (use the library from HW2 to solve the QP).
7. Implement a function that computes the amount of constraint violation for the nonlinear constraints (not the linearized one), i.e. the sum of the absolute values of all the constraints (i.e. assuming constraints of the form $c(x) = 0$ we want to compute $|c(x)|$).
8. Implement a filter linear search to test if a step should be accepted. You will implement the (simplified) filter line search explained below.
9. Terminate the algorithm when you either reached the maximum number of iterations (e.g. 100) or when the KKT optimality conditions are satisfied (within a small margin, e.g. $10^{-4}$).

Once you have a solution, make sure to check that it satisfies the constraints! You can display the robot using the ```animate_robot``` function.

#### (Simple) filter linear search
Once you have a potential step $\textrm{step}_x$, you need to find a step $\alpha$ to update your guess of the solution $x_{guess}$. We will accept a step that either reduces the amount of constraint violation or reduces the cost.

Let's denote $f(x)$ the cost at $x$ and $|c(x)|$ the amount of constraint violation at $x$. Initialize the variable $f_{best} = \infty$ and $c_{best}=\infty$ at the beginning of the SQP. 

Then do the following during the line search.
1. Set $\rho$ to a number between 0 and 1 (e.g. 0.5) and set $\alpha = 1$
2. If $f(x_{guess} + \alpha \textrm{step}_x) < f_{best}$ then set $f_{best} \leftarrow f(x_{guess} + \alpha \textrm{step}_x)$ and accept the step

   Or 

   If $|c(x_{guess} + \alpha \textrm{step}_x)| < c_{best}$ then set $c_{best} \leftarrow |c(x_{guess} + \alpha \textrm{step}_x)|$ and accept the step
3. If the step was not accepted set $\alpha \leftarrow \rho \alpha$ and go back to Step 2.
4. If the step was accepted update the guess $x_{guess} \leftarrow x_{guess} + \alpha \textrm{step}_x$

## Part 2 - Model predictive control (MPC) (50 points)
1. Use the trajectory optimization method from Part I to design a MPC controller and test it using the simulator below (you may need to use a shorter horizon). In particular, verify that it can handle perturbations by calling the ```quadrotor.simulate``` function with ```disturbance = True``` (when setting disturbance to True, the simulator will generate a random perturbation every 1 second). Simulate your controller for 10 seconds, plot the state anc control evolution.
2. Explain your intended design in the report, including the cost function and found control law

The ```quadrotor.simulate``` function takes as an input an initial state, a controller, the number of discrete time steps and a boolean value to indicate the presence of perturbation. The controller has to be a function taking as an input a state and time index and outputting a control vector.

## Bonus (10 points)
Add a state constraint to perform the looping while maintening a positive altitude. Use the origin as an initial state.

In [2]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.animation as animation
import IPython

def controller(x, t):
    return np.zeros(2)

x_init = np.array([0, 0, 0., 0 ,0, 0])
horizon_length = 100
t, state, u = quadrotor.simulate(x_init, controller, horizon_length, disturbance=True)
quadrotor.animate_robot(state, u)

<source type="video/mp4" src="data:video/mp4;base64,AAAAIGZ0eXBNNFYgAAACAE00ViBpc29taXNvMmF2YzEAAAAIZnJlZQAAMBFtZGF0AAACrwYF//+r
3EXpvebZSLeWLNgg2SPu73gyNjQgLSBjb3JlIDE2NCByMzEwOCAzMWUxOWY5IC0gSC4yNjQvTVBF
Ry00IEFWQyBjb2RlYyAtIENvcHlsZWZ0IDIwMDMtMjAyMyAtIGh0dHA6Ly93d3cudmlkZW9sYW4u
b3JnL3gyNjQuaHRtbCAtIG9wdGlvbnM6IGNhYmFjPTEgcmVmPTMgZGVibG9jaz0xOjA6MCBhbmFs
eXNlPTB4MzoweDExMyBtZT1oZXggc3VibWU9NyBwc3k9MSBwc3lfcmQ9MS4wMDowLjAwIG1peGVk
X3JlZj0xIG1lX3JhbmdlPTE2IGNocm9tYV9tZT0xIHRyZWxsaXM9MSA4eDhkY3Q9MSBjcW09MCBk
ZWFkem9uZT0yMSwxMSBmYXN0X3Bza2lwPTEgY2hyb21hX3FwX29mZnNldD0tMiB0aHJlYWRzPTE1
IGxvb2thaGVhZF90aHJlYWRzPTIgc2xpY2VkX3RocmVhZHM9MCBucj0wIGRlY2ltYXRlPTEgaW50
ZXJsYWNlZD0wIGJsdXJheV9jb21wYXQ9MCBjb25zdHJhaW5lZF9pbnRyYT0wIGJmcmFtZXM9MyBi
X3B5cmFtaWQ9MiBiX2FkYXB0PTEgYl9iaWFzPTAgZGlyZWN0PTEgd2VpZ2h0Yj0xIG9wZW5fZ29w
PTAgd2VpZ2h0cD0yIGtleWludD0yNTAga2V5aW50X21pbj0yNSBzY2VuZWN1dD00MCBpbnRyYV9y
ZWZyZXNoPTAgcmNfbG9va2FoZWFkPTQwIHJjPWNyZiBtYnRyZWU9MSBjcmY9MjMuMCBxY29tcD0w
LjYwIHFwbWluPTAgcXBtYXg9NjkgcXBzdGVwPTQgaXBfcmF0aW89MS40MCBhcT0xOjEuMDAAgAAA
ERJliIQAN//+9vD+BTY7mNCXEc3onTMfvxW4ujQ3vc4AAAMAAAMAAAMAADSY0H4zd2rDhiAAAAMA
JgAGCANgAzAEdwBJhifrPwALwRFkBJwPzJUmFGT2qHXo4m6RCitIlhRYet3pioe7PtHMnO4JkntJ
ujXVK9mpvpov1Dq5CrLp26+E/qiy2XJ1DXbFQFIeD6dj4d/Ycg4MlRjSj5F/jO4dneZbKchBkxER
1+dwaLseoQTUOIHe+AjMd/01j3lkHunyumHGATF5QfuGMNEhmAiiUAYOIeHxFiIgg2yx/sREsJlD
kHggnz8ibjDHy6sod15ejTD7nM3nSKn356x7NU+rLjiy+TGV7zlKZBPivMIavE++a9A0lqsJYdgr
ySjKMHT/BXnDAgmDXfO64jpiyGPFAqtkFWJ5TEQleZ1J5ApIiAbrmqAKw3p8+/i/wJs8O+8pdk8P
HVf783WDYB8jTEDGGwrhpmxJeLpr1ICYHqwOTW5Bd1WdwtukKLg3p6qWT7+2vbqd9rIiSvziYtq8
25i0R3TvRrRr1mzOMZ8C1Cu9K0kb2sxSkg1soFJwsR/nsWc/EvC3H0L+5eWapvOtxtKl6rGVIaX0
FIWpc7KaJRRckMG2H0O5eoaJtHM76WDdRTxhaPitkh9CQGq91Pwe5RR/6jlcnbr7cslcoSuhyftW
3iRyNKEtESY1euYcpcxKSDVz5/DYbmNXDqBcJctYkdbAuvtm/25snj+jhMpakm793/30LpXeFCUv
Z2UCe1WuDf9vvh/rXBCN7W5KLQ/HhZr/5fknJz8utFADVUyvxw82ViPMfucXpJoo+8k+VwCbfpjQ
Mu9obn1ljubDGHzsiEv40BfYAsQggZUxfi5e4mEa3OXj5TEXcu8bdqoXP1XEkZVn2fCUDErAFJQ9
2Af6mKmjoCMz+Hq46Cd5ugBZpJUnHDJrQtC89ypszYX7cgATaorUoMoGgRhg4WWNjZz9p+gKMXg8
ponBbMeat8Sqo0eclegFuqp+eZpzbmoE2wQlSIuopU1QqWkVxhMlGZSHSzbW+iHaVoC8s3BPj/y5
/fqYLXzBnpz+uJlc57q4y9l2liNww8qv496xPZWVh+F/+kITP/VbZikw9pQ/eMUo3VOn6iENgPHC
XDEZLgM+OiYibwQJbhacZExgQWjinv4UBScPcq5B6/MK4qlNIfZPGC9DjFdmdgujaffKxIJKPOMy
OZVa2hZIvE7ZoAs2xF/Iby0Y2rOvYJNIfGzqm9vjeVZxKvoc8i2W+dCD1uLfp6enX6l609SVLqc8
hcxa6On5Q4BvipBP6AXmv+URURZtpxZKswbwDJvEyqsssyYA9DeRg+EtnYYkWugLqw6Yopz58aQh
eIKxkigoXlF6WLqojCAdgISEKwB3h5O/AAADAAADAAADAACSG6A0yKZI1qpdRDNiVCyPsxPzKVot
mk4XssNGmD+9Im7lohH1forjZjL6QikXCfQDTTqhaaQ+jZal899v0ViilI4xQ59SZ49fgl0cRRZM
X7Qxutm+LRQc420l9bFZ3AcyCSNQaZWbWN6zV9bIs+xmxMSuCzHRY4+0q0rqClmk0m2ZQ5QOoAsg
KSIuvC/RKpCGhkq2yMhdIfFiANj75Hso1fta0zXYm8H2R/1PPzhHArlYxusNVZJFgDrLk1NqMf8/
wz8dWsVvuNhXDohG4RDoJYac1KN9D5zd+BoAoDrAbMmRYEDE/6IbrZ4gCR9vcldP1h21Bz1hAjR6
Q7k+04mwlDZboLCbqUAFRoux1DazSN/4VlPK/4oGTTT5bCgLGDiMvQcxv+IoYC9Uj64YILKhG569
EbrNO3L/pTWsM/7cu409piXKbVUdRYLkE0j0r1PVPUHMgdTF3nofNmAMDEhfNxMJKHc8Dn5NKm4Z
cYrovq84GSXXpmB3zSXoWyuAAAjs2+2m4x6bUHJVqPIM/lBd2c7xEfdJACHRtI2uCu/vne7lmtD4
WdvjG5QIlq1yEioS8IrYsQsUReDqauOCO/Q8Bm1aoH65dG/FgMwWLEdfktD4mU5lYtKGdRB0to75
bULSK7o9a6k4BGxinUpUos3edJwrJX1y/vGnrTXkVrt6o6qWGbUesXTl1ce3/yCvAbyHHDG6wkA/
i9la/f6g0m8qz4dQddSQjpcIErC7oYXbDc66z1L3VPYSaxiP1V0PAmuBV9/1H97XuUdzKfruZRju
fFUopIvrmEWH2vh3hq/w/7xtTLyFACqMnlORUjC8HyxnkQ8dAdmyNSisQrTWF2AIFVP/snenBob5
XtvI7qpIXf4MnWjYeBssaRk+O/r1e4RxV1f9RbWjiFb1Lt+l33OT7TWhW04ry0iwRPhxbLov6Y1I
mfyn3G8P/USUmNUsSV1zbtFWo/PWdIhJx2jzm4QV7DN0DjbqxEJLMEopM83lKDSbVqHkn5109ARZ
rB1I88+jlSVaQ440/6gtH9J6DxQ4tV9Xp97tfPl5LtZb+NgfXb0rc4C74b5BEL3gWLCqGFUhaZsx
pl5pRL3DeqYZqZgzM51u9vCGTfABsMV6t+c6L1zMS7FjgdUPKgQMqrUjEGrRo5Wb5TpD/1B1K6EX
oOhv5kagAAADAAAEUmwepvyCSNhFPI9EEEFFgs+MqXLj8NL5cb1P5fTyEpr3LrivckcS1m90+5gz
jSJx8Cc9aN2p3UBm7p16XqRWrqS5prJCe08V2pGgKKU7GDLAi0DyVWUMU6SWMl4heIALm0lVfDNy
9TuPsiRa6BrsHhsA1CShbEmF8Hp9O/fOQjkkpTOJkMNcCjvRIFX7mPP6UeaRsbMj8Rg/NkTrq/mi
Gd7RlEpEYaEErsR0E5Jj9tGUUUz41Q1X0nPk+QsZPhO5nRlfdJsT+eQxCk//vrcuo/krF97SzPKK
xUx2nC9BXsTmCLNElIbZs